In [2]:
import glob
import sys
import numpy as np

from enterprise.pulsar import Pulsar
from enterprise.signals import signal_base
from enterprise.signals import parameter
from enterprise.signals import white_signals
from enterprise.signals import gp_signals

# import customized files to introduce 2T modes.
import gp_bases_new as gp_bases
import pl_prior as gp_priors

from enterprise.signals import selections

import multiprocess
from PTMCMCSampler.PTMCMCSampler import PTSampler

Cannot import PINT? Meh...
Do not have mpi4py package.


In [2]:
def get_pulsar_noise(pta , ret, ro): 

        ndim = len(pta.params)

                
        groups0 = [[i,i+1] for i in range(0,ndim-1,2)]        
        groups0.extend([range(ndim)])

        groups1 = [range(ndim)]
        outDir0='/home/xuex/DPDM-PPTA/noise_chain/first_run/'+pta.pulsars[0]
        outDir1='/home/xuex/DPDM-PPTA/noise_chain/second_run/'+pta.pulsars[0]

        if ro==False:
                x0 = np.zeros(ndim)
                x0=np.hstack([par.sample() for par in pta.params])
                cov0 = np.diag(np.ones(ndim)*0.5)
                
                sampler = PTSampler(ndim, pta.get_lnlikelihood, pta.get_lnprior, 
                        cov0, groups=groups0 , outDir = outDir0, verbose=True)
                print(pta.pulsars[0]+'***starts')
                sampler.sample(x0, 50000,isave=1000)
                chain0 = np.loadtxt(outDir0+'/chain_1.txt')



                x1 = chain0[np.where(chain0==np.max(chain0[:,-3]))[0][0],:-4]
                cov1 = np.load(outDir0 + '/cov.npy')
                sampler = PTSampler(ndim, pta.get_lnlikelihood, pta.get_lnprior, 
                        cov1 , groups=groups1 , outDir=outDir1, verbose=True)
                sampler.sample(x1, 100000, isave=1000)

        chain1 = np.loadtxt(outDir1+'/chain_1.txt')

        # End of the second run.


        # Return the ln-likelihood value of the best fit(maximal likelihood).

        MLHselect = chain1[np.where(chain1==np.max(chain1[:,-3]))[0][0],:]
        Dict = {pta.params[i].name:MLHselect[i] for i in range(ndim)}
        ret.value = (Dict,pta.get_lnlikelihood(Dict),pta.get_lnprior(Dict))
        print(pta.pulsars[0]+'***finished')


        # End of the function.

In [3]:
datadir = 'DR2'
parfiles = sorted(glob.glob(datadir + '/newpars/*.par'))
timfiles = sorted(glob.glob(datadir + '/toas/*.tim'))

In [ ]:
psrs=[]
for ipsr in range(len(parfiles)):
    psr = Pulsar(parfiles[ipsr], timfiles[ipsr], ephem="DE436",clk="TT(BIPM2018)")
    psrs.append(psr)
    print(psr.name)

In [6]:

# red noise
nmodes = 30


log10_A = parameter.Uniform(-21,-9)
gamma = parameter.Uniform(0,7)
pl = gp_priors.powerlaw(log10_A=log10_A, gamma=gamma)
dm_basis = gp_bases.createfourierdesignmatrix_dm(nmodes=nmodes)
red_basis = gp_bases.createfourierdesignmatrix_red(nmodes=nmodes)
selection = selections.Selection(selections.by_band)

dmn = gp_signals.BasisGP(pl, dm_basis, name='dm', coefficients=False)
spn = gp_signals.BasisGP(pl, red_basis, name='sp',coefficients=False)
bdn = gp_signals.BasisGP(pl, red_basis, name='bd',coefficients=False,selection=selection)

# white noise
backend = selections.Selection(selections.by_backend)
efac = parameter.Uniform(0.01, 10.0)
equad = parameter.Uniform(-8.5, -5)

ef = white_signals.MeasurementNoise(efac=efac,selection=backend)
eq = white_signals.EquadNoise(log10_equad=equad,selection=backend)
wnb = ef + eq

# timing model
tm = gp_signals.TimingModel()

In [7]:
model0  = tm + wnb + dmn + spn
model1  = tm + wnb + dmn + spn + bdn
ptas=[]
for psr in psrs:
        if psr.name in ['J0437-4715','J1939+2134']:
                pta=signal_base.PTA( model1(psr) )
        else:
                pta=signal_base.PTA( model0(psr) )
        ptas.append(pta)
        print(psr.name)

J0437-4715
J0613-0200
J0711-6830
J1017-7156
J1022+1001
J1024-0719
J1045-4509
J1125-6014
J1446-4701
J1545-4550
J1600-3053
J1603-7202
J1643-1224
J1713+0747
J1730-2304
J1732-5049
J1744-1134
J1824-2452A
J1832-0836
J1857+0943
J1909-3744
J1939+2134
J2124-3358
J2129-5721
J2145-0750
J2241-5236


In [8]:
jobs = []
RETs={}        
for i in range(len(psrs)):
        RETs[i] = multiprocess.Manager().Value('i',0)
        p = multiprocess.Process(target=get_pulsar_noise, args=(ptas[i],RETs[i],False))
        jobs.append(p)
        p.start()
for p in jobs:
        p.join()


# Return the sum of the Maximal Likelihood values.

MLHselect = [RET.value for RET in RETs.values()]

J0437-4715***starts
J0613-0200***starts
J0711-6830***starts
J1017-7156***starts
J1022+1001***starts
J1024-0719***starts


/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))
/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))
/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


J1045-4509***starts
J1125-6014***starts


/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))
/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))
/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


J1446-4701***starts


...

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/PTMCMCSampler-2015.2-py3.7.egg/PTMCMCSampler/PTMCMCSampler.py", line 1026, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 170, in __call__
    TNTs = self.pta.get_TNT(params)
Traceback (most recent call last):
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 265, in get_TNT
    return [signalcollection.get_TNT(params) for signalcollection in self._signalcollections]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 170, in __call__
    TNTs = self.pta.get_TNT(params)
Process Process-33:
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/home/xuex/anaconda3/lib/python3

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 262, in <listcomp>
    return [signalcollection.get_TNr(params) for signalcollection in self._signalcollections]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 295, in get_lnlikelihood
    return self._lnlikelihood(params, **kwargs)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "<ipython-input-2-724ce43e788f>", line 33, in get_pulsar_noise
    sampler.sample(x1, 100000, isave=1000)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/PTMCMCSampler-2015.2-py3.7.egg/PTMCMCSampler/PTMCMCSampler.py", line 490, in PTMCMCOneStep
    newlnlike = self.logl(y)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 900, in wrapper
    cache[k

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 842, in get_TNT
    return Nvec.solve(T, left_array=T)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/PTMCMCSampler-2015.2-py3.7.egg/PTMCMCSampler/PTMCMCSampler.py", line 1026, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 295, in get_lnlikelihood
    return self._lnlikelihood(params, **kwargs)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 265, in <listcomp>
    return [signalcollection.get_TNT(params) for signalcollection in self._signalcollections]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 962, in solve
    mult = np.dot(left_array.T, mult)
  File "<ipytho

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 900, in wrapper
    cache[key] = func(self, params)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 842, in get_TNT
    return Nvec.solve(T, left_array=T)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 962, in solve
    mult = np.dot(left_array.T, mult)
  File "<__array_function__ internals>", line 6, in dot
KeyboardInterrupt
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 900, in wrapper
    cache[key] = func(self, params)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 900, in wrapper
    cache[key] = func(self, params)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 265, in <listcomp>
    return [signalcollection.get_TNT(params) for signalcollection in self._signalcollections]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 792, in get_ndiag
    ndiags = [signal.get_ndiag(params) for signal in self._signals]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 792, in <listcomp>
    ndiags = [signal.get_ndiag(params) for signal in self._signals]
KeyboardInterrupt
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 900, in wrapper
   

In [ ]:
# Parameter dictionary, save the best fit noise parameters.
'''
Dict = {}
for x in MLHselect:
        Dict.update(x[0])

np.save('noise_chain/noisepars_m'+str(nmodes),Dict)
'''